In [1]:
import pandas as pd
from keras.src.backend.jax.math import top_k

In [2]:
books = pd.read_csv("predicted_categories.csv")

In [4]:
books

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_category_mapping
0,9780002005883,0002005883,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nature
4,9780006280934,0006280935,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Travel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3754,9788122200850,8122200850,Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
3755,9788173031014,8173031010,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Fiction
3756,9788179921623,817992162X,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Non-fiction
3757,9788185300535,8185300534,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Non-fiction


#### Top k retruns the top k results

In [14]:
from transformers import pipeline
pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base",top_k =None, return_all_scores=True)
# from transformers import pipeline
# pipe = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")


Device set to use cpu


In [12]:
pipe("i love you")

[[{'label': 'anger', 'score': 0.0025094228330999613},
  {'label': 'disgust', 'score': 0.0003477402788121253},
  {'label': 'fear', 'score': 0.0008826187695376575},
  {'label': 'joy', 'score': 0.9639525413513184},
  {'label': 'neutral', 'score': 0.0012242394732311368},
  {'label': 'sadness', 'score': 0.027516981586813927},
  {'label': 'surprise', 'score': 0.0035664220340549946}]]

In [15]:
pipe(books["description"][0])

[[{'label': 'fear', 'score': 0.6548399925231934},
  {'label': 'neutral', 'score': 0.1698525995016098},
  {'label': 'sadness', 'score': 0.11640939861536026},
  {'label': 'surprise', 'score': 0.02070068009197712},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161462128162384},
  {'label': 'anger', 'score': 0.003935154061764479}]]

#### Getting the emotional tone of each sentence in the description

In [28]:
predictions = pipe(books["description"][0].split("."))

In [18]:
# splits description into multiple sentences
sentences = books["description"][0].split(".")

In [77]:
sorted(pipe(sentences[2])[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.011031905189156532},
 {'label': 'disgust', 'score': 0.043422672897577286},
 {'label': 'fear', 'score': 0.014084091410040855},
 {'label': 'joy', 'score': 0.014211490750312805},
 {'label': 'neutral', 'score': 0.6462159156799316},
 {'label': 'sadness', 'score': 0.242733433842659},
 {'label': 'surprise', 'score': 0.02830057218670845}]

In [23]:
import numpy as np

In [78]:
emotion_labels = ["anger","disgust","fear","joy","neutral","sadness","surprise"]

In [99]:
isbn = []

emotion_scores = {label:[] for label in emotion_labels}

def calculate_emotion_score(predictions):

    per_emotions_score = {label:[] for label in emotion_labels}

    for prediction in  predictions:

        # sorted by name|label, to preserve the order of the labels
        sorted_prediction = sorted(prediction, key=lambda x: x["label"])

        for index, label in enumerate(emotion_labels):

            per_emotions_score[label].append(sorted_prediction[index]["score"])

    # print(per_emotions_score)

    return {labels : np.max(scores) for labels, scores in per_emotions_score.items()}

In [ ]:
### Refernece code
# for i in range(0,10):
#
#     isbn.append(books["isbn13"][i])
#     sentences = books["description"][i].split(".")
#
#     predictions = pipe(sentences)
#
#     max_scores = calculate_emotion_score(predictions)
#
#     # print(max_scores)
#
#     for label, scores in max_scores.items():
#
#         emotion_scores[label].append(max_scores[label])
#
#
# print(emotion_scores)


In [102]:
from tqdm import tqdm

for i in tqdm(range(len(books))):

    isbn.append(books["isbn13"][i])

    sentences = books["description"][i].split(".")

    predictions = pipe(sentences)

    max_scores = calculate_emotion_score(predictions)

    # print(max_scores)

    for label, scores in max_scores.items():

        emotion_scores[label].append(max_scores[label])


print(emotion_scores)


# per_emotions_score

100%|██████████| 3759/3759 [04:05<00:00, 15.29it/s]

{'anger': [0.0641336515545845, 0.6126194000244141, 0.0641336515545845, 0.3514845371246338, 0.0814124271273613, 0.23222433030605316, 0.5381841063499451, 0.0641336515545845, 0.0641336515545845, 0.3006698489189148, 0.0641336515545845, 0.6126194000244141, 0.0641336515545845, 0.3514845371246338, 0.0814124271273613, 0.23222433030605316, 0.5381841063499451, 0.0641336515545845, 0.0641336515545845, 0.3006698489189148, 0.0641336515545845, 0.0641336515545845, 0.6115756034851074, 0.10621592402458191, 0.0641336515545845, 0.07239263504743576, 0.09586359560489655, 0.1621590256690979, 0.0641336515545845, 0.08193826675415039, 0.22346076369285583, 0.10715220868587494, 0.0641336515545845, 0.0641336515545845, 0.0641336515545845, 0.39636674523353577, 0.06472676247358322, 0.0641336515545845, 0.0641336515545845, 0.0641336515545845, 0.3029327690601349, 0.06683336198329926, 0.0641336515545845, 0.0641336515545845, 0.0641336515545845, 0.1688966304063797, 0.0641336515545845, 0.0641336515545845, 0.0641336515545845

In [108]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [109]:
books = pd.merge(books, emotions_df, how="left", on="isbn13")

In [110]:
books

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_category_mapping,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602
1,9780002005883,0002005883,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602
2,9780002261982,0002261987,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252546
3,9780002261982,0002261987,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252546
4,9780006178736,0006178731,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3764,9788122200850,8122200850,Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction,0.541745,0.226290,0.305919,0.040564,0.594006,0.111690,0.078765
3765,9788173031014,8173031010,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Fiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
3766,9788179921623,817992162X,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Non-fiction,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,0.057625
3767,9788185300535,8185300534,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Non-fiction,0.064134,0.104007,0.459269,0.759456,0.951104,0.368110,0.078765


In [111]:
books.to_csv("book_with_emotions.csv", index=False)

In [113]:
books["large_thumbnail"] = books["thumbnail"] + "&fife=w800"
books["large_thumbnail"] = np.where(books["large_thumbnail"].isna(),"cover_not_found.jpg",books["large_thumbnail"])

In [114]:
books["simple_category_mapping"].unique()

array(['Fiction', 'Nature', 'Travel', 'Art', 'Games & Activities',
       'Non-fiction', 'Sports & Recreation', 'Music', 'Photography',
       'Cooking', 'Gardening', 'Crafts & Hobbies', 'Animals',
       'Meditation'], dtype=object)